# Strategy for Automatic Detection and Segmentation of Microcalcifications in Mammograms Using Image Processing and Lightweight Algorithms

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
pip install pydicom

## Import libraries

In [ ]:
# Import python libraries
import imageio as io
import matplotlib.pyplot as plt
import numpy as np
import os
from skimage import io
import pandas as pd
import cv2
from PIL import Image
from skimage import data
from skimage.filters import threshold_multiotsu
import matplotlib
from scipy.stats import linregress
import math
import random
from skimage import io, color, filters, data, exposure
from typing_extensions import assert_type
import copy
from pydicom import dcmread
from scipy.stats import linregress
import math
import random
from skimage import io, color, filters, data, exposure
import time
from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt
from math import sqrt
from skimage.feature import blob_dog, blob_log, blob_doh
from skimage.filters import threshold_otsu



## FUNCTIONS

In [ ]:
# Start time
start_time = time.time()


# Open the image in pgm format
def openImage(rute):
  image= io.imread(rute)
  return image

# Image Size and Pixel Value Range
def sizeImage(image):
    """Prints the dimensions and the minimum and maximum pixel values of the image."""
    print(f'Image dimensions: {image.shape}')
    print(f'Minimum pixel value: {image.min()}')
    print(f'Maximum pixel value: {image.max()}')


# I normalize the cropped image and with Logarithmic tranf. I scale it from 0 to 1
def normalize_image(img):
    img_norm = (img - img.min()) / (img.max() - img.min())
    return img_norm


def plotImageTittle2(image, title, cmap = 'gray', size=100):
    fig, ax = plt.subplots(dpi=size)
    ax.imshow(image,cmap=cmap)
    plt.title(str(title))
    plt.show


# Plot an image with  title
def plotImageTittle(image, title, size=100):
    fig, ax = plt.subplots(dpi=size)
    ax.imshow(image,cmap='gray')
    plt.title(str(title))
    plt.show

# Plot two horizontal images with title
def plotImageTwoTittle(image1, image2, title1, title2, size=100):
    fig, ax = plt.subplots(1, 2, dpi=size)
    # First sub-graph
    ax[0].imshow(image1, cmap='gray')
    ax[0].set_axis_off()
    ax[0].set_title(str(title1))
    # Second sub-graph
    ax[1].imshow(image2, cmap='gray')
    ax[1].set_axis_off()
    ax[1].set_title(str(title2))

    plt.show()


#  Plot three horizontal images with title
def plotImageThreeTittle(image1, image2, image3, title1, title2, title3, size=400):
    fig, ax = plt.subplots(1,3, dpi=size)
    ax[0].imshow(image1, cmap='gray')
    ax[0].set_axis_off()
    ax[0].set_title(str(title1), fontsize=5)

    ax[1].imshow(image2, cmap='gray')
    ax[1].set_axis_off()
    ax[1].set_title(str(title2), fontsize=5)

    ax[2].imshow(image3, cmap='gray')
    ax[2].set_axis_off()
    ax[2].set_title(str(title3), fontsize=5)
    plt.show()

# Plot ten horizontal images with title


# Define custom colormap
custom_cmap = ListedColormap(["blue", "green", "yellow", "red", "white"])  # Ajusta colores si querés

# Function to plot 10 images with titles
def plotImageTenTittle2(img1, img2, img3, img4, img5, img6, img7, img8, img9, img10, t1, t2, t3, t4, t5, t6, t7, t8, t9, t10):
    fig, axes = plt.subplots(2, 5, figsize=(18, 6), dpi=200)
    fig.subplots_adjust(hspace=0.1, wspace=0)

    imgs = [img1, img2, img3, img4, img5, img6, img7, img8, img9, img10]
    titles = [t1, t2, t3, t4, t5, t6, t7, t8, t9, t10]

    for i, (img, title) in enumerate(zip(imgs, titles)):
        ax = axes[i // 5, i % 5]

        if i == 3:  # Cuarta imagen (índice 3) - Multi-Otsu
            ax.imshow(img, cmap=custom_cmap)
        elif i == len(imgs) - 1:
            ax.imshow(img, cmap="seismic")
        else:
            ax.imshow(img, cmap="gray")

        ax.axis('off')
        ax.set_title(str(title), fontsize=10)

    plt.show()



# Plot a histogram - range=[0, 1]
def histogram(img, n_bins, title=''):
    h = img.ravel()
    _, nbins = np.histogram(h, bins=n_bins, range=[0, 1])

    plt.hist(h, bins=nbins, edgecolor='black')
    plt.title(format(title))
    #plt.grid()
    plt.show()


 # Plot two histograms - range=[0, 1]
def hyper_histogram(hyp1, hyp2, n_bins, title1='', title2=''):

    h1 = hyp1[:,:].ravel()
    h2 = hyp2[:,:].ravel()
    _, nbins = np.histogram(h1, bins=n_bins, range=[0, 1])

    fig, axs = plt.subplots(1, 2, figsize=(12, 6))
    axs[0].hist(h1, bins=nbins, edgecolor='black')
    axs[0].set_title(format(title1))
    plt.grid()
    axs[1].hist(h2, bins=nbins, edgecolor='black')
    axs[1].set_title(format(title2))
    plt.grid()
    plt.show

# visualize image and histogram - range=[0, 1]
def visualize_imag_histogram(image, n_bins=256, size=(12, 6), dpi=80, image_title='Grayscale Image', hist_title='Luminance Histogram'):
    # Create a figure with two subplots: one for the image and one for the histogram
    fig, axs = plt.subplots(1, 2, figsize=size, dpi=dpi)

    # Display the grayscale image
    axs[0].imshow(image, cmap='gray')
    axs[0].set_title(image_title)
    axs[0].axis('off')  # Hide the axes of the image

    # Calculate and plot the histogram in the second subplot
    axs[1].hist(image.ravel(), bins=n_bins, range=[0, 1], edgecolor='black')
    axs[1].set_title(hist_title)
    axs[1].set_xlabel('Luminance')
    axs[1].set_ylabel('Frequency')
    axs[1].grid(True)

    plt.tight_layout()
    plt.show()



# Crop images- DATASET: Mias
def area_crop(image,x,y,r):
    print('Coordinates for crop:',x ,",",y,",",r)
    img_copy = np.copy(image)
    y = image.shape[0] - y
    img_recorte = img_copy[y-r:y+r, x-r:x+r]
    return img_recorte


# In this function the origin of coordinates is in the lower left corner
# DATASET: Mias

def draw_area(img,x, y, r):
    #Image origin at (x=0, y=maximo_y)
    print('Coordinates:',x ,",",y,",",r)
    y = img.shape[0] - y
    img_copy = np.copy(img)
    cv2.line(img_copy,(x-r,y-r),(x+r,y-r),(255,0,0),4)
    cv2.line(img_copy,(x-r,y+r),(x+r,y+r),(255,0,0),4)
    cv2.line(img_copy,(x-r,y-r),(x-r,y+r),(255,0,0),4)
    cv2.line(img_copy,(x+r,y-r),(x+r,y+r),(255,0,0),4)
    plt.imshow(img_copy, cmap='gray')
    #return img_copy



# Draw a rectangle with four coordinates - dcm format
def affected_area_dcm(img5, x1, y1, x2, y2, x3, y3, x4, y4):
    img5_copy = np.copy(img5)

    # Color in RGB format (255, 87, 51) corresponding to #FF5733
    color = (255, 87, 51)

    # Draw the rectangle lines in the specified color
    cv2.line(img5_copy, (x1, y1), (x2, y2), color, 5)
    cv2.line(img5_copy, (x2, y2), (x3, y3), color, 5)
    cv2.line(img5_copy, (x3, y3), (x4, y4), color, 5)
    cv2.line(img5_copy, (x4, y4), (x1, y1), color, 5)

    return img5_copy


# Function to CROP A mammographic - dcm format
def image_crop_dcm(image, y1, y4, x1, x2):
    image_crop = image[y1:y4, x1:x2]
    return image_crop


# from RGB to YIQ
def rgb_to_y(matrix_rgb):
    matrix_rgb = np.array(matrix_rgb)
    coefficients = np.array([0.299, 0.587, 0.114])
    matrix_y = np.dot(matrix_rgb, coefficients)
    return matrix_y


# Define a function to compute the optimal exponent factor based on pixel intensity distribution
def calculate_optimal_exponent(image, percentile=90, lower_bound=2, upper_bound=20, scaling_factor=0.97):
    # Calculate the specified percentile of pixel intensities
    intensity_percentile = np.percentile(image, percentile / 100.0 * np.max(image))

    # Suggest an exponent based on the computed percentile
    suggested_n = lower_bound + (upper_bound - lower_bound) * (1 - intensity_percentile)

    # Apply the scaling factor to adjust the suggested exponent
    suggested_n_new = suggested_n * scaling_factor
    print(f"Original suggested exponent: {suggested_n}")
    print(f"Using only {scaling_factor * 100}% of the suggested exponent. The new suggested exponent is: {suggested_n_new}")

    # Ensure the new exponent is within the specified range
    return max(lower_bound, min(upper_bound, suggested_n_new))


# Function to apply Multi-Otsu and decompress the thresholds
def multi_Otsu(img_power, dpi=100):
    plt.rcParams['font.size'] = 9
    image = img_power
    thresholds = threshold_multiotsu(image)
    otsu_threshold1, otsu_threshold2 = thresholds
    print("The Multi-Otsu threshold is:", otsu_threshold1)
    print("The Multi-Otsu threshold is:", otsu_threshold2)
    regions = np.digitize(image, bins=thresholds)

    # Create subplots with the specified dpi
    fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(10, 3.5), dpi=dpi)
    ax[0].imshow(image, cmap='gray', clim=(0, 1))
    ax[0].set_title('Power Image')
    ax[0].axis('off')
    ax[1].hist(image.ravel(), bins=30)
    ax[1].set_title('Histogram')
    for thresh in thresholds:
        ax[1].axvline(thresh, color='r')
    ax[2].imshow(regions, cmap='jet')
    ax[2].set_title('Multi-Otsu (three types)')
    ax[2].axis('off')
    plt.subplots_adjust()
    plt.show()

    return regions, otsu_threshold1, otsu_threshold2

# === BI-RADS 4 Classification Function ===
def classify_suspicion_level(density):
    """
    Classifies the suspicion level of a lesion based on its estimated malignancy risk percentage.
    """
    if density > 2 and density <= 10:
        return (
            "Category 4A: Low but sufficient suspicion (>2% to 10%).\n"
        )
    elif density > 10 and density <= 50:
        return (
            "Category 4B: Moderate suspicion (>10% to 50%).\n"
        )
    elif density > 50 and density < 95:
        return (
            "Category 4C: High suspicion (>50% to <95%).\n"
        )
    else:
        return (
            "Outside BI-RADS 4A–4C range.\n"
            "- Either benign (<2%) or extremely suspicious (>95%).\n"
            "- Further assessment or different category may apply.\n"
        )

# After using the conventional Multi-Otsu, 4 classes are obtained in the mammogram according to their luminance


def multi_Otsu_4zonesv2(img_power, dpi=100):
    """
    This function performs a multi-level Otsu segmentation to divide a breast image
    into three zones based on intensity thresholds and calculates key metrics for
    breast density analysis and potential microcalcification detection.

    Parameters:
        img_power (ndarray): Input image normalized to [0, 1].
        dpi (int): Resolution for the output visualization plots.

    Returns:
        regions (ndarray): Segmented image regions.
        otsu_threshold1 (float): First Otsu threshold.
        otsu_threshold2 (float): Second Otsu threshold.
        interval_min (float): Lower bound of intensity interval for microcalcifications.
        interval_max (float): Upper bound (fixed at 1.0).
    """
    plt.rcParams['font.size'] = 9

    # === Apply Multi-Otsu Thresholding ===
    image = img_power
    thresholds = threshold_multiotsu(image)
    otsu_threshold1, otsu_threshold2 = thresholds
    print("Multi-Otsu Threshold 1:", otsu_threshold1)
    print("Multi-Otsu Threshold 2:", otsu_threshold2)

    max_pixel_value = np.max(image)
    print("Maximum pixel value:", max_pixel_value)

    # === Segment the image into 3 zones based on thresholds ===
    zone_1_pixels = image[image < otsu_threshold1]
    zone_2_pixels = image[(image >= otsu_threshold1) & (image < otsu_threshold2)]
    zone_3_pixels = image[image >= otsu_threshold2]

    # === Calculate area (number of pixels) in each zone ===
    area_zone_1 = len(zone_1_pixels)
    area_zone_2 = len(zone_2_pixels)
    area_zone_3 = len(zone_3_pixels)
    area_total = area_zone_1 + area_zone_2 + area_zone_3

    # === Metric 2.1: Dense Breast Proportion (PDM-Z3) ===
    if (area_zone_2 + area_zone_3) > 0:
        pdm_z3 = (area_zone_3 / (area_zone_2 + area_zone_3)) * 100
    else:
        pdm_z3 = 0

    print(f"➤ Dense Breast Proportion (PDM-Z3): {pdm_z3:.2f}%")
    print(classify_suspicion_level(pdm_z3))

    # === Metric 2.2: Proportion of Luminance in Dense Tissue (PLTD-Z3) ===
    sum_luminance_zone_2 = np.sum(zone_2_pixels)
    sum_luminance_zone_3 = np.sum(zone_3_pixels)
    if (sum_luminance_zone_2 + sum_luminance_zone_3) > 0:
        pltd_z3 = (sum_luminance_zone_3 / (sum_luminance_zone_2 + sum_luminance_zone_3)) * 100
    else:
        pltd_z3 = 0

    print(f"➤ Proportion of Luminance in Dense Tissue (PLTD-Z3) 💡: {pltd_z3:.2f}%")

    # === Metric 2.3: Luminance Averages in Dense Region (PL-Z3) ===
    avg_zone_3 = np.mean(zone_3_pixels) if len(zone_3_pixels) > 0 else 0
    alpha = 6
    weights = np.exp(alpha * zone_3_pixels)
    avg_weighted_zone_3 = (np.sum(zone_3_pixels * weights) / np.sum(weights)) if np.sum(weights) > 0 else 0

    print(f"✨ Classical luminance average in dense zone (PL-Z3 classical): {avg_zone_3:.2f}")
    print(f"📈 Exponentially weighted luminance average in dense zone (PL-Z3 weighted): {avg_weighted_zone_3:.2f}")

    # === Metric 2.4: Probable Intensity Range for Microcalcifications (RIPM) ===
    if avg_weighted_zone_3 * 1.15 > 1.0:
        interval_min = 1.0
    else:
        interval_min = avg_weighted_zone_3 * 1.15
    interval_max = 1.0

    print(f"➤🚨 Probable Intensity Range for Microcalcifications (RIPM): [{interval_min:.2f}, {interval_max:.2f}]")

    if interval_min > interval_max:
        avg_intensity_interval = 0
    else:
        zone_3_in_interval = zone_3_pixels[(zone_3_pixels >= interval_min) & (zone_3_pixels <= interval_max)]
        avg_intensity_interval = np.mean(zone_3_in_interval) if len(zone_3_in_interval) > 0 else 0

    print(f"💥 Average Intensity within RIPM Interval: {avg_intensity_interval:.2f}")

    # === Visualization of segmentation and histogram ===
    regions = np.digitize(image, bins=thresholds)
    mask_interval = (image >= interval_min) & (image <= interval_max)
    segmentation_colors = plt.cm.jet(regions / regions.max())
    segmentation_colors[mask_interval] = [1, 1, 1, 1]

    fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(10, 3.5), dpi=dpi)
    ax[0].imshow(image, cmap='gray')
    ax[0].set_title('Power Image')
    ax[0].axis('off')

    ax[1].hist(image.ravel(), bins=30)
    ax[1].set_title('Intensity Histogram')
    for idx, thresh in enumerate(thresholds):
        ax[1].axvline(thresh, color='r', linestyle='--', label=f'Threshold {idx+1}: {thresh:.2f}')
    ax[1].axvline(interval_min, color='g', linestyle='--', label=f'Interval Min: {interval_min:.2f}')
    ax[1].axvline(interval_max, color='g', linestyle='--', label=f'Interval Max: {interval_max:.2f}')
    ax[1].legend(loc='upper right')

    ax[2].imshow(segmentation_colors)
    ax[2].set_title('Segmentation with MC Highlighted in White')
    ax[2].axis('off')

    plt.tight_layout()
    plt.show()

    # === Summary Table (Metrics 2.1 to 2.4) ===
    metrics = {
        "Metric": [
            "Dense Breast Proportion (PDM-Z3) (%)",
            "Proportion of Luminance in Dense Tissue (PLTD-Z3) (%)",
            "Classical Luminance Average in Dense Zone (PL-Z3 classical)",
            "Exponentially Weighted Luminance Average in Dense Zone (PL-Z3 weighted)",
            "Probable Intensity Range for Microcalcifications (RIPM)",
            "Average Intensity within RIPM Interval"
        ],
        "Value": [
            f"{pdm_z3:.2f}",
            f"{pltd_z3:.2f}",
            f"{avg_zone_3:.2f}",
            f"{avg_weighted_zone_3:.2f}",
            f"[{interval_min:.2f}, {interval_max:.2f}]",
            f"{avg_intensity_interval:.2f}"
        ]
    }

    df_metrics = pd.DataFrame(metrics)
    print("\n**Summary of Metrics:**")
    print(df_metrics)

    multiotsu_results = {
        'PDM_Z3': round(pdm_z3, 3),
        'PLTD_Z3': round(pltd_z3, 3),
        'PL_Z3_classical': round(avg_zone_3, 3),
        'PL_Z3_weighted': round(avg_weighted_zone_3, 3),
        'RIPM_min': round(interval_min, 3),
        'RIPM_max': round(interval_max, 3),
        'RIPM_avg': round(avg_intensity_interval, 3)
    }

    return segmentation_colors, regions, otsu_threshold1, otsu_threshold2, interval_min, interval_max, multiotsu_results



# Function to create a bar chart and a table for pixel count and percentage
def plot_and_table(image, regions, otsu_threshold1, otsu_threshold2, dpi=100):
    # Define the masks for each class
    mask_class1 = (regions == 0)
    mask_class2 = (regions == 1)
    mask_class3 = (regions == 2)

    # Calculate the number of pixels in each class
    count_class1 = np.count_nonzero(mask_class1)
    count_class2 = np.count_nonzero(mask_class2)
    count_class3 = np.count_nonzero(mask_class3)

    # Total number of pixels
    total_pixels = regions.size

    # Calculate the percentage of pixels in each class
    percent_class1 = (count_class1 / total_pixels) * 100
    percent_class2 = (count_class2 / total_pixels) * 100
    percent_class3 = (count_class3 / total_pixels) * 100

    # Calculate the mean and standard deviation for each class using the original image
    mean_class1 = np.mean(image[mask_class1])
    std_class1 = np.std(image[mask_class1])
    mean_class2 = np.mean(image[mask_class2])
    std_class2 = np.std(image[mask_class2])
    mean_class3 = np.mean(image[mask_class3])
    std_class3 = np.std(image[mask_class3])

    # Define the intervals for each class
    intervals = [
        f"(0, {otsu_threshold1:.6f})",
        f"({otsu_threshold1:.6f}, {otsu_threshold2:.6f})",
        f"({otsu_threshold2:.6f}, 1)"
    ]

    # Create a DataFrame for the table
    data = {
        'Class': ['Class 1 (Blue)', 'Class 2 (Green)', 'Class 3 (Red)'],
        'Value Range': intervals,
        'Pixel Count': [count_class1, count_class2, count_class3],
        'Pixel Percentage (%)': [round(percent_class1, 6), round(percent_class2, 6), round(percent_class3, 6)],
        'Mean Value': [f"{mean_class1:.6f}", f"{mean_class2:.6f}", f"{mean_class3:.6f}"],
        'Standard Deviation': [f"{std_class1:.6f}", f"{std_class2:.6f}", f"{std_class3:.6f}"]
    }
    df = pd.DataFrame(data)

    # Adjust pandas display settings to show 6 decimal places
    pd.set_option('display.float_format', '{:.6f}'.format)

    # Print the table
    print("\nResults Table:")
    print(df)

    # Create a bar chart for the pixel count per class
    fig, ax = plt.subplots(dpi=dpi)
    ax.bar(df['Class'], df['Pixel Count'], color=['blue', 'green', 'red'])
    ax.set_title('Pixel Count per Class')
    ax.set_xlabel('Class')
    ax.set_ylabel('Pixel Count')
    plt.tight_layout()
    plt.show()

    return df



# opening filter
def opening_filter(image):
    # Realiza la operación de apertura morfológica
    kernel = np.ones((2,2), np.uint8)  # Definir un kernel de 3x3
    opening = cv2.morphologyEx(image, cv2.MORPH_OPEN, kernel)
    return opening  # Asegúrate de retornar el resultado


# Anscombe transformation
def anscombe(img):
    img_anscombe= 2.0*np.sqrt(img + 3.0/8.0)
    sizeImage(img_anscombe)
    return(img_anscombe)


# Intensity using the cosine function
def cosine_function(img):
    img_cosine= 1-np.cos((np.pi/2)*(img/255.0))
    sizeImage(img_cosine)
    return(img_cosine)


# DoG filter function (difference of gaussians)
def DoG_filter(img, k1, k2):
    # apply a gaussian blur with a 3x3 kernel to create the lower-frequency component
    dog_low = cv2.GaussianBlur(img, (3, 3), 0)

    # apply a gaussian blur with a 5x5 kernel to create the higher-frequency component
    dog_high = cv2.GaussianBlur(img, (5, 5), 0)

    # calculate the difference of gaussians (DoG) by combining the two blurred images with specified weights
    dog = k1 * dog_low - k2 * dog_high

    # return the resulting dog image
    return dog



# Change colors of Otsu thresholding (black to white)
def apply_otsu_and_plot(image, title, size=100):
    # Copiar la imagen normalizada
    image_copy = copy.copy(image)

    # Calcular el umbral de Otsu
    otsu_threshold = threshold_otsu(image)

    # Aplicar el umbral de Otsu
    image_copy[image <= otsu_threshold] = 1
    image_copy[image > otsu_threshold] = 0

    # Retornar la imagen procesada
    return image_copy



# Overlays an RGB color on an image based on a segmentation mask
def painting_segmented_end(img, mask, color_rgb=[57, 255, 0]):
    """
    Overlays an RGB color on an image based on a segmentation mask.

    Parameters:
    img (numpy array): Input grayscale image to be processed.
    mask (numpy array): Binary mask where 0 indicates the regions to be colored.
    color_rgb (list, optional): RGB color to apply to the segmented regions.
                                Default is [57, 255, 0] (light green).

    Returns:
    numpy array: RGB image with the segmented regions painted in the specified color.
    """

    # Normalize the input image to the range [0, 1]
    img = normalize_image(img)

    # Convert the grayscale image to an RGB format by stacking it across three channels
    img_rgb = np.stack((img, img, img), axis=-1)

    # Scale the image to the range [0, 255] and convert it to an 8-bit unsigned integer
    img_rgb = img_rgb * 255.0
    img_rgb = img_rgb.astype(np.uint8)

    # Apply the specified RGB color to the regions where the mask is 0
    img_rgb[mask == 0, 0] = color_rgb[0]  # Red channel
    img_rgb[mask == 0, 1] = color_rgb[1]  # Green channel
    img_rgb[mask == 0, 2] = color_rgb[2]  # Blue channel

    return img_rgb




# F - blobs_with_visualization

# F - blobs_with_visualization


# F - blobs_with_visualization

def analyze_blobs_with_visualization(imag_power, pixel_to_mm=0.2, max_sigma=10):
    """
    Detects and classifies blobs in the given image using LoG, DoG, and DoH methods.
    Displays the detections and prints blob counts and average diameters per size category.
    Also shows bar plots with summary statistics.
    """
    from math import sqrt
    import numpy as np
    import matplotlib.pyplot as plt
    from skimage.feature import blob_log, blob_dog, blob_doh

    # ---------------------------------------
    # BLOB DETECTION
    # ---------------------------------------
    blobs_log = blob_log(imag_power, min_sigma=0.9, max_sigma=max_sigma, threshold=0.09)
    blobs_log[:, 2] = blobs_log[:, 2] * sqrt(2)

    blobs_dog = blob_dog(imag_power, min_sigma=0.9, max_sigma=max_sigma, threshold=0.09)
    blobs_dog[:, 2] = blobs_dog[:, 2] * sqrt(2)

    blobs_doh = blob_doh(imag_power, min_sigma=0.9, max_sigma=max_sigma, threshold=0.09)

    blobs_list = [blobs_log, blobs_dog, blobs_doh]
    titles = ['LoG', 'DoG', 'DoH']

    categories = {
        "d < 0.1 mm": lambda d: d < 0.1,
        "0.1 ≤ d ≤ 1.5 mm": lambda d: 0.1 <= d <= 1.5,
        "d > 1.5 mm": lambda d: d > 1.5
    }

    blob_counts = {det: {cat: 0 for cat in categories} | {'total': 0} for det in titles}
    blob_diameters = {det: {cat: [] for cat in categories} for det in titles}

    # ---------------------------------------
    # VISUALIZATION
    # ---------------------------------------
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))

    for idx, (blobs, title) in enumerate(zip(blobs_list, titles)):
        axes[idx].set_title(title)
        axes[idx].imshow(imag_power, cmap="gray")
        img_height, img_width = imag_power.shape

        for blob in blobs:
            y, x, r = blob
            radius_mm = r * pixel_to_mm
            diameter_mm = 2 * radius_mm

            for label, condition in categories.items():
                if condition(diameter_mm):
                    if label == "d < 0.1 mm":
                        color = 'blue'
                    elif label == "0.1 ≤ d ≤ 1.5 mm":
                        color = 'red'
                    else:
                        color = 'yellow'

                    blob_counts[title][label] += 1
                    blob_counts[title]['total'] += 1
                    blob_diameters[title][label].append(diameter_mm)
                    break

            # Draw circle
            circle = plt.Circle((x, y), r, color=color, linewidth=1.5, fill=False)
            axes[idx].add_patch(circle)

            # Position label
            offset = 5
            text_y = y - r - offset
            if text_y < 10:
                text_y = y + r + offset
            if text_y > img_height - 10:
                text_y = img_height - 10

            text_x = max(10, min(x, img_width - 10))

            if (10 <= text_x <= img_width - 10) and (10 <= text_y <= img_height - 10):
                axes[idx].text(
                    text_x, text_y, f"{diameter_mm:.2f} mm",
                    color=color, fontsize=6, fontweight='bold', ha="center"
                )

        axes[idx].set_xlim(0, img_width)
        axes[idx].set_ylim(img_height, 0)
        axes[idx].set_axis_off()

    plt.subplots_adjust(wspace=0.05, hspace=0.05)
    plt.show()

    results = {}
    results['Detector'] = []
    results['Total_blobs'] = []
    results['Blobs_candidates'] = []
    results['percent_medium_candidates'] = []
    results['Average_diameter_candidates'] = []

    # ---------------------------------------
    # OUTPUT SUMMARY
    # ---------------------------------------
    print("\n📊 Blob count and average diameter by detector and size range:")
    for detector in titles:
        total = blob_counts[detector]['total']
        medium = blob_counts[detector]["0.1 ≤ d ≤ 1.5 mm"]
        percent_medium = (medium / total * 100) if total > 0 else 0
        blob_counts[detector]['medium_percent'] = percent_medium  # Added to dictionary
        results['Detector'].append(detector)
        results['Total_blobs'].append(total)

        print(f"\n🧠 {detector}:")
        for cat in categories:
            count = blob_counts[detector][cat]
            diameters = blob_diameters[detector][cat]
            average = np.mean(diameters) if diameters else 0
            if cat == "0.1 ≤ d ≤ 1.5 mm":
                results['Blobs_candidates'].append(count)
                results['percent_medium_candidates'].append(round(percent_medium, 3))
                results['Average_diameter_candidates'].append(round(average, 3))

            print(f"  {cat}: {count} blobs | average diameter = {average:.3f} mm")
        print(f"  TOTAL: {total} blobs")
        print(f"  ✅ 0.1–1.5 mm blobs = {medium} ({percent_medium:.1f}%)")

    # ---------------------------------------
    # BAR CHART SUMMARY
    # ---------------------------------------
    total_counts = [blob_counts[det]['total'] for det in titles]
    avg_radii = []
    count_medium = []  # Count of blobs with 0.1 ≤ d ≤ 1.5 mm

    for det in titles:
        all_diameters = []
        for diam_list in blob_diameters[det].values():
            all_diameters.extend(diam_list)
        avg_radius = np.mean(all_diameters) / 2 if all_diameters else 0
        avg_radii.append(avg_radius)

        count_medium.append(blob_counts[det]["0.1 ≤ d ≤ 1.5 mm"])

    fig, ax1 = plt.subplots(figsize=(10, 6))
    bar_width = 0.25
    index = np.arange(len(titles))

    # First axis: Total blobs and medium blobs
    bars1 = ax1.bar(index, total_counts, bar_width, label='Total Blobs', color='skyblue')
    bars2 = ax1.bar(index + bar_width, count_medium, bar_width, label='0.1–1.5 mm Blobs', color='lightgreen')
    ax1.set_ylabel('Blob Count')
    ax1.set_xticks(index + bar_width / 2)
    ax1.set_xticklabels(titles)
    ax1.set_title('Blob Detection Summary')

    # Second axis: Avg radius
    ax2 = ax1.twinx()
    bars3 = ax2.bar(index + 2 * bar_width, avg_radii, bar_width, label='Avg. Radius (mm)', color='orange')
    ax2.set_ylabel('Avg. Radius (mm)')

    # Legends
    ax1.legend(loc='upper left')
    ax2.legend(loc='upper right')

    plt.tight_layout()
    plt.show()

    return blob_counts, blob_diameters, results



# EXECUTION

In [ ]:
def procces(path, img_files, data_file):


    multiotsu_general_results = {}
    multiotsu_general_results['Image_name'] = []
    multiotsu_general_results['Tissue'] = []
    multiotsu_general_results['Abnormality'] = []
    multiotsu_general_results['Severity'] = []
    multiotsu_general_results['PDM_Z3'] = []
    multiotsu_general_results['PLTD_Z3'] = []
    multiotsu_general_results['PL_Z3_classical'] = []
    multiotsu_general_results['PL_Z3_weighted'] = []
    multiotsu_general_results['RIPM_min'] = []
    multiotsu_general_results['RIPM_max'] = []
    multiotsu_general_results['RIPM_avg'] = []

    blobs_general_results = {}
    blobs_general_results['Image_name'] = []
    blobs_general_results['Tissue'] = []
    blobs_general_results['Abnormality'] = []
    blobs_general_results['Severity'] = []
    blobs_general_results['Detector'] = []
    blobs_general_results['Total_blobs'] = []
    blobs_general_results['Blobs_candidates'] = []
    blobs_general_results['percent_medium_candidates'] = []
    blobs_general_results['Average_diameter_candidates'] = []

    df_data = pd.read_csv(path + data_file, delimiter=';')
    print('keys', df_data.keys())
    for i, f in enumerate(img_files):
      if i < 370: ###### Control de cantidad de imagenes a ejecutar
        print(f'Processing image: {f}')
        img_route = path + f
        img_name = img_route.split('/')[-1]
        img_name = img_name.split('.')[0]
        # Extraer coordenadas del archivo de datos para la imagen
        df_filter = df_data[df_data.Name == img_name]
        if df_filter.empty:
          print(f"No se encontraron coordenadas para la imagen: {img_name}")
        else:
          #print(f'Processing image: {f}')
          x = df_filter.Coordinate_x.values[0]
          y = df_filter.Coordinate_y.values[0]
          r = df_filter.Radius.values[0]
          # extract data
          tissue = df_filter.Tissue.values[0]
          abnormality = df_filter.Abnormality.values[0]
          severity = df_filter.Severity.values[0]
          # Abrir y graficar imagen
          initial_img = openImage(img_route)
          plotImageTittle(initial_img, "Mammography", size=100)
          sizeImage(initial_img)
          #Dibujar zona de interes y hacer recorte de la zona
          img_box = draw_area(initial_img, x, y, r)
          imag_crop= area_crop(initial_img, x, y, r)
          #Normalizacion de la imagen
          imag_crop_norm = normalize_image(imag_crop)
          #Filtro de potencia
          optimal_n = calculate_optimal_exponent(imag_crop_norm, percentile=90, lower_bound=2, upper_bound=20, scaling_factor=0.15)
          imag_power = imag_crop_norm**optimal_n
          #Multiotsu
          imag_multiOtsu, otsu_threshold1, otsu_threshold2 = multi_Otsu(imag_power, dpi=100)
          df = plot_and_table(imag_power, imag_multiOtsu, otsu_threshold1, otsu_threshold2, dpi=70)
          #Multiotsu 4 zonas
          img_fiveotsu, regions, _, _ , interva_max, interval_max, multiotsu_results = multi_Otsu_4zonesv2(imag_power)
          imag_multiOtsu_copy1 = copy.copy(imag_power)
          intervalo_min, intervalo_max =interva_max, interval_max  #0.72, 1  # Reemplaza con los valores reales de tu función multi_Otsu3
          print(" Probable intensity interval for microcalcifications from multi_Otsu_4zones:","(", interva_max ,";",interval_max,")")
          # Apply thresholding based on the probable interval
          imag_multiOtsu_copy1[imag_multiOtsu_copy1 < intervalo_min] = 0
          imag_multiOtsu_copy1[imag_multiOtsu_copy1 >= intervalo_min] = 1
          # Display the resulting image
          imag_multiOtsu_norm = normalize_image(imag_multiOtsu_copy1)
          # Opening Filter
          imag_opening = opening_filter(imag_multiOtsu_copy1).astype(np.float32)  # Convert to float32 if necessary
          imag_opening_norm = normalize_image(imag_opening)
          #Convolution between the Otsu mask refined with the opening filter and the image corresponding to the mammogram crop
          imag_crop_norm_copy = copy.copy(imag_crop_norm)
          imag_crop_norm_copy[imag_opening_norm == 0] = 0
          # Anscombe transformation
          imag_anscombe = anscombe(imag_crop_norm_copy)
          imag_anscombe_norm = normalize_image(imag_anscombe)
          # Cosine function
          imag_cosine= cosine_function(imag_anscombe_norm)
          imag_cosine_norm = normalize_image(imag_cosine)
          # DoG filter
          img_dog = DoG_filter(imag_cosine_norm, k1=1.1, k2=0.9)
          img_dog_norm = normalize_image(img_dog)
          # Final image
          final_image= img_dog_norm*imag_crop_norm_copy
          final_image_norm = normalize_image(final_image)
          #Change colors of Otsu thresholding (black to white)
          imag_dog_norm_copy = apply_otsu_and_plot(img_dog_norm, "end", size=100)
          # Segmented microcalcification
          copy_crop= copy.copy(imag_crop_norm)
          copy_crop_rgb = painting_segmented_end(img=copy_crop, mask=imag_dog_norm_copy, color_rgb=[255, 0, 128])
          # All images
          plotImageTenTittle2(initial_img, imag_crop_norm, imag_power, img_fiveotsu, imag_opening_norm,
                              imag_anscombe_norm, imag_cosine_norm, img_dog_norm,  final_image_norm, copy_crop_rgb,
                              'Mammography','Crop','Power','MultiOtsu','Opening','Anscombe',
                              'Cosine','DoG','Seg microcalcification','Colormaps')
          # Analyze blobs with visualization
          counts, diameters, blobs_results = analyze_blobs_with_visualization(imag_power)

          # Save data in dictionary
          multiotsu_general_results['Image_name'].append(img_name)
          multiotsu_general_results['Tissue'].append(tissue)
          multiotsu_general_results['Abnormality'].append(abnormality)
          multiotsu_general_results['Severity'].append(severity)
          multiotsu_general_results['PDM_Z3'].append(multiotsu_results['PDM_Z3'])
          multiotsu_general_results['PLTD_Z3'].append(multiotsu_results['PLTD_Z3'])
          multiotsu_general_results['PL_Z3_classical'].append(multiotsu_results['PL_Z3_classical'])
          multiotsu_general_results['PL_Z3_weighted'].append(multiotsu_results['PL_Z3_weighted'])
          multiotsu_general_results['RIPM_min'].append(multiotsu_results['RIPM_min'])
          multiotsu_general_results['RIPM_max'].append(multiotsu_results['RIPM_max'])
          multiotsu_general_results['RIPM_avg'].append(multiotsu_results['RIPM_avg'])

          for idx in range(len(blobs_results['Detector'])):
            blobs_general_results['Image_name'].append(img_name)
            blobs_general_results['Tissue'].append(tissue)
            blobs_general_results['Abnormality'].append(abnormality)
            blobs_general_results['Severity'].append(severity)
            blobs_general_results['Detector'].append(blobs_results['Detector'][idx])
            blobs_general_results['Total_blobs'].append(blobs_results['Total_blobs'][idx])
            blobs_general_results['Blobs_candidates'].append(blobs_results['Blobs_candidates'][idx])
            blobs_general_results['percent_medium_candidates'].append(blobs_results['percent_medium_candidates'][idx])
            blobs_general_results['Average_diameter_candidates'].append(blobs_results['Average_diameter_candidates'][idx])

    return multiotsu_general_results, blobs_general_results




In [ ]:
def main(path):
    """ This method makes general validations and begins the
    #	the process of the meshes found. """

    files = os.listdir(path)
    img_files = [x for x in files if ".pgm" in x]
    img_files = sorted(img_files)
    data_file = [x for x in files if "Data_base_mias.csv" in x][0]

    # Checking existing files to process
    if len(files) > 0:
      print("Number of images found: {}".format(len(files)))
      multiotsu_general_results, blobs_general_results = procces(path, img_files, data_file)
      df_multiotsu = pd.DataFrame(multiotsu_general_results)
      df_blobs = pd.DataFrame(blobs_general_results)
      # Save results
      df_multiotsu.to_csv(path + 'Results_mias_multiotsu.csv', index=False, sep=';')
      df_blobs.to_csv(path + 'Results_mias_blobs2.csv', index=False, sep=';')
      #print(df_multiotsu.head(10))
      #print(df_blobs.head(10))
    else:
      print( "Data folder doesn't contain any '.pgm' files to process...")
    return 0

In [ ]:
path = '/content/drive/MyDrive/projet_uns/mamografias_mias_2025/'
main(path)